# Pandas Express
# TTC-Delays project
Team Members: 

Gaspare Castagna

Amirhossein Heidari

Avery Hoffer

Michael Metallo

In [5]:
import numpy as numpy
import pandas as pd
import os
import numpy as np


Import, read, concat bus delay data

In [6]:
def read_excel_sheets(file_path):
    xls = pd.ExcelFile(file_path)
    data_frames = []
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name)
        data_frames.append(df)
    return pd.concat(data_frames, ignore_index=True)

In [7]:
df_2014 = read_excel_sheets('bus_2014.xlsx')
df_2015 = read_excel_sheets('bus_2015.xlsx') 
df_2016 = read_excel_sheets('bus_2016.xlsx') 
df_2017 = read_excel_sheets('bus_2017.xlsx') 
df_2018 = read_excel_sheets('bus_2018.xlsx')
df_2018 = df_2018.iloc[:, :-1]
df_2019 = read_excel_sheets('bus_2019.xlsx')
df_2019 = df_2019.iloc[:, :-3]
df_2020 = read_excel_sheets('bus_2020.xlsx')
df_2020 = df_2020.iloc[:, :-3]
df_2020.rename(columns={'Gap': 'Min Gap', 'Delay': 'Min Delay'}, inplace=True)
df_2021 = read_excel_sheets('bus_2021.xlsx')
df_2021.rename(columns={'Date': 'Report Date'}, inplace=True)
df_2021 = df_2021.iloc[:, :-3]
df_2022 = read_excel_sheets('bus_2022.xlsx')
df_2022.rename(columns={'Date': 'Report Date'}, inplace=True)

bus_delay_list = [df_2014,df_2015,df_2016,df_2017,df_2018,df_2019,df_2020,df_2021,df_2022]
df_bus_delay = pd.concat(bus_delay_list)

df_bus_delay['YTime'] = pd.to_datetime(df_bus_delay['Time'], format='%H:%M:%S', errors='coerce').dt.strftime('%H:%M')
df_bus_delay['DateTime'] = df_bus_delay['Report Date'].dt.strftime('%Y-%m-%d')+'-'+ df_bus_delay['YTime']
df_bus_delay['DateTime'] = pd.to_datetime(df_bus_delay['DateTime'])

drop_columns = ['YTime','Time','Report Date','Direction']
df_bus_delay = df_bus_delay.drop(drop_columns, axis = 1)
df_bus_delay = df_bus_delay.dropna(subset=['Route'])

df_bus_delay['copy'] = df_bus_delay['Location'].str.lower()
df_bus_delay['copy'] = df_bus_delay['copy'].str.replace('vp','victoria park')
df_bus_delay['copy'] = df_bus_delay['copy'].str.replace(' to ','&').str.replace('b/w','').str.replace('w/b','').str.replace('e/b','').str.replace('s/b','').str.replace('n/b','')
df_bus_delay['copy'] = df_bus_delay['copy'].str.replace('stn','station').str.replace(' and ','&').str.replace('/','&').str.replace(' ','')
df_bus_delay['copy'] = df_bus_delay['copy'].str.replace('(','').str.replace(')','')
df_bus_delay['copy'] = df_bus_delay['copy'].str.replace('@','&')
df_bus_delay['copy'] = df_bus_delay['copy'].astype(str)
df_bus_delay = df_bus_delay[~df_bus_delay['copy'].str.isdigit()]
df_bus_delay['copy'] = df_bus_delay['copy'].apply(lambda x: '&'.join(sorted(x.split('&'))) if x.count('&') == 1 else x)

stops = pd.read_csv('stops.txt')
stops['cleaned_stops'] = stops['stop_name'].str.replace('at', '&').str.replace('Rd', '').str.replace('Ave', '').str.replace('Dr', '').str.replace('St', '')
stops['cleaned_stops'] = stops['cleaned_stops'].str.replace('Side', '').str.replace('@','&')
stops['cleaned_stops'] = stops['cleaned_stops'].str.replace(' ', '').str.lower()
stops['cleaned_stops'] = stops['cleaned_stops'].str.replace('(north)', '').str.replace('(south)', '').str.replace('(east)', '').str.replace('(west)', '')
stops['cleaned_stops'] = stops['cleaned_stops'].str.replace('(mall)', '')
stops['cleaned_stops'] = stops['cleaned_stops'].apply(lambda x: re.sub(r'\(.*?\)', '', x))
stops['cleaned_stops'] = stops['cleaned_stops'].astype(str).str.replace('\d+', '', regex=True)
stops['cleaned_stops'] = stops['cleaned_stops'].apply(lambda x: '&'.join(sorted(x.split('&'))) if x.count('&') == 1 else x)

df_bus_delay_fuzz = df_bus_delay[~df_bus_delay['copy'].str.contains('route')].copy()
df_bus_delay_fuzz = df_bus_delay_fuzz[~df_bus_delay_fuzz['copy'].isin(stops['cleaned_stops'])]
df_bus_delay_fuzz = df_bus_delay_fuzz[df_bus_delay_fuzz['copy'].str.contains('&')]

unique_values = pd.unique(df_bus_delay_fuzz['copy'])

# Create a new DataFrame with a column of unique values
df_delay_fuzz = pd.DataFrame({
    'fuzz_locations': unique_values
})

In the section below I am testing the fuzz score function we will be using:

This function takes in a string "location" and compares it to each entry in a list of strings "entry". 
It returns a string corresponding to the "best" match for the input string from the list of strings.
We can adjust the threshold to avoid false positives. Strings with no matches above the threshold will not be modified.

In [26]:
from rapidfuzz import process
def find_closest_match_rapid(row, reference_list, threshold = 50):
    
    match = process.extractOne(row, reference_list)
    if match[1] < threshold:
        return "check"
    else:
        return match[0]

In [19]:
len(df_delay_fuzz)

37547

In [27]:
df_delay_fuzz['fuzzed'] = df_delay_fuzz['fuzz_locations'].apply(find_closest_match_rapid, args=(stops['cleaned_stops'],))
df_delay_fuzz.to_csv('df_bus_delay_with_threshold.csv')

In [28]:
df_fuzz_threshold = pd.read_csv('df_bus_delay_with_threshold.csv')
df_fuzz = pd.read_csv('df_bus_delay_no_threshold.csv')

In [54]:
bus_delays = pd.merge(df_bus_delay, df_fuzz_threshold, left_on='copy', right_on='fuzz_locations', how='left')

# Create the 'analysis' column and fill it with 'fuzz' if 'location' is not NaN, otherwise 'not fuzzed'
bus_delays['analysis'] = bus_delays['fuzzed'].where(~bus_delays['fuzz_locations'].isna(), bus_delays['copy'])

columns_to_drop = ['fuzz_locations', 'fuzzed','analysis location','Unnamed: 0']
bus_delays = bus_delays.drop(columns=columns_to_drop)

In [55]:
bus_delays.head(50)

,Route,Day,Location,Incident,Min Delay,Min Gap,Vehicle,DateTime,copy,analysis
0,95,Wednesday,York Mills station,Mechanical,10.0,20.0,1734.0,2014-01-01 00:23:00,yorkmillsstation,yorkmillsstation
1,102,Wednesday,Entire run for route,General Delay,33.0,66.0,8110.0,2014-01-01 00:55:00,entirerunforroute,entirerunforroute
2,54,Wednesday,lawrence and Warden,Mechanical,10.0,20.0,7478.0,2014-01-01 01:28:00,lawrence&warden,warden
3,112,Wednesday,Kipling Station,Emergency Services,18.0,36.0,8084.0,2014-01-01 01:30:00,kiplingstation,kiplingstation
4,24,Wednesday,VP and Ellesmere,Investigation,10.0,20.0,7843.0,2014-01-01 01:37:00,ellesmere&victoriapark,ellesmere&victoriapark
5,129,Wednesday,Scarborough Town,Mechanical,10.0,20.0,1755.0,2014-01-01 01:50:00,scarboroughtown,scarboroughtown
6,36,Wednesday,humberline at woodlot,Mechanical,16.0,32.0,1322.0,2014-01-01 02:28:00,humberlineatwoodlot,humberlineatwoodlot
7,53,Wednesday,Passmore and Markham,Mechanical,18.0,36.0,7043.0,2014-01-01 02:30:00,markham&passmore,markham&passmore
8,36,Wednesday,finch stn,Mechanical,16.0,30.0,7933.0,2014-01-01 02:33:00,finchstation,finchstation
9,320,Wednesday,steele at yonge,Mechanical,7.0,14.0,8187.0,2014-01-01 02:44:00,steeleatyonge,steeleatyonge


In [56]:
bus_delays.to_csv('bus_delays.csv')

In [ ]:
df_bus_delay.head(50)